In [1]:
# Import library yang diperlukan
import numpy
import io
from itertools import permutations
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

read training file and create 

In [2]:
# Membaca file dan proses init table
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0].lower()+'|'+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>|'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+'|'+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [3]:
# Memanggil fungsi init table yang dibangun dari dataset yang menghasilkan tag_count, word_tag, dan tag_trans

tag_count, word_tag, tag_trans = read_file_init_table('corpus.tsv')

In [4]:
# Fungsi untuk membangun tabel transition probability
def create_trans_prob_table(tag_trans, tag_count):
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            trans_idx = tag1+'|'+tag2
            if trans_idx in tag_trans:
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [5]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)

In [6]:
# Fungsi untuk membangun tabel emission
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split('|')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_key = current_word+'|'+current_tag
        emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [7]:
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [8]:
# Fungsi untuk mencari tag dari suatu kata pada kalimat menggunakan viterbi.

def viterbi(trans_prob, emission_prob, tag_count, sentence):
    #initialization
    viterbi_mat = {}
    tag_sequence = []
    
    max_before = {}
    for tag in tag_count:
        max_before[tag] = 1
    
    sentence_words = sentence#.split()
    for i in range(0,len(sentence_words)):
        # print(sentence_words[i])
        word_check = sentence_words[i]
        for tag in tag_count:
            max_curr = 0
            for tag2 in tag_count:
                # Kalkulasi viterbi, nilai probability dari kata sebelumnya * transisi * emisi
                cal = max_before.get(tag2, 1) * trans_prob.get( tag + '|'+  tag2, 1) * emission_prob.get(sentence_words[i] + '|' + str(tag), 1)
                if (max_curr < cal):
                    max_curr = cal
            max_before[tag] = max_curr
            
            
        max_output = max_before.copy()
        max_value = {}
        max_value['max'] = 0
        max_value['tag'] = ''
        max_value['word'] = word_check
        # mencari nilai maksimum probability suatu kata pada setiap kata pada kalimat.
        for j in max_output:
            if ((max_output[j] < 1) and (max_output[j] > max_value['max'])  ):
                max_value['max'] = max_output[j]
                max_value['tag'] = j
        # probability tertinggi merupakan tag dari kata tersebut
        tag_sequence.append(max_value)
        
    # Mengembalikan tag dari setiap kata dalam bentuk list
    return tag_sequence

In [9]:
# Membaca data test
def read_datatest_viterbi(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

In [10]:
#load datatest
sentences, tags = read_datatest_viterbi('dataset_test.tsv')
tags_persentence = []

# lowercase semua kata pada datatest
for sentence in range(len(sentences)):
    for word in range(len(sentences[sentence])):
        sentences[sentence][word] = sentences[sentence][word].lower()
        
# mulai penghitungan viterbi dengan memanggil fungsi viterbit
for i in range(len(sentences)) :
    tags_predict = viterbi(trans_prob, emission_prob, tag_count, sentences[i])
    tags_persentence.append(tags_predict)
   

In [11]:
# persiapkan y_test yang merupakan tag asli dari kata
y_test = []
i = 0
for tagss in tags:
    for tag in tagss:
        y_test.append(tag)
        i+= 1
        
print(i)

270


In [12]:
# persiapkan y_predict yang merupakan tag prediksi dari model viterbi suatu kata
i = 0
y_predict = []
for tags in tags_persentence:
    for tag in tags:
        y_predict.append(tag)
        i+= 1
        
print(i)

270


In [13]:
correct = 0
for i in range(len(y_test)):
    if y_test[i] == y_predict[i]['tag']:
        correct += 1
print('Akurasi viterbi: ',correct / len(y_predict) * 100)

# print(y_predict)

Akurasi viterbi:  94.07407407407408


In [14]:
y_predict

[{'max': 0.004003874717468518, 'tag': 'NN', 'word': 'hal'},
 {'max': 0.28908002991772624, 'tag': 'PR', 'word': 'ini'},
 {'max': 0.011218605237449972, 'tag': 'VB', 'word': 'terjadi'},
 {'max': 0.048470948012232416, 'tag': 'SC', 'word': 'karena'},
 {'max': 0.0010655473038424282, 'tag': 'NN', 'word': 'pelaku pasar'},
 {'max': 0.0012290045063498567, 'tag': 'VB', 'word': 'mencari'},
 {'max': 3.228931223764934e-05, 'tag': 'NN', 'word': 'untung'},
 {'max': 0.0012684989429175475, 'tag': 'FW', 'word': 'gain'},
 {'max': 0.013091830510065225, 'tag': 'IN', 'word': 'atas'},
 {'max': 0.005521472392638037, 'tag': 'NN', 'word': 'kenaikan'},
 {'max': 0.010025831449790119, 'tag': 'NN', 'word': 'rupiah'},
 {'max': 0.43027522935779816, 'tag': 'SC', 'word': 'yang'},
 {'max': 0.035896389965327354, 'tag': 'RB', 'word': 'cukup'},
 {'max': 0.00462772521596051, 'tag': 'JJ', 'word': 'tajam'},
 {'max': 0.5490947735985121, 'tag': 'Z', 'word': ','},
 {'max': 0.0031197806699650205, 'tag': 'VB', 'word': 'ujar'},
 {'m

# BASELINE

In [15]:
# Fungsi untuk mencari semua tag yang ada
def getTagThatPossible(word):
    collection = []
    for item in emission_prob:
        if (item.split('|')[0] == word):
            collection.append(item.split('|')[1])
    return collection

In [16]:
# Model baseline dengan mencari kemunculan tag terbanyak suatu kata
def findTag(word):
    possibleTag = getTagThatPossible(word)
    maxProb = 0
    maxTag = ''
    for item in possibleTag:
        key = word + '|' + item
        if (emission_prob.get(key, 0) > maxProb):
            maxTag = item 
    return maxTag

In [17]:
sentences, tags = read_datatest_viterbi('dataset_test.tsv')
tags_predict = []

# mulai penghitungan baseline

for sentence in range(len(sentences)):
    for word in range(len(sentences[sentence])):
        word =  sentences[sentence][word].lower()
        tags_predict.append(findTag(word))        

y_predict = tags_predict

In [18]:
# Persiapkan data test untuk menghitung akurasi
y_test = []
i = 0
for tagss in tags:
    for tag in tagss:
        y_test.append(tag)
        i+= 1
        
print(i)

270


In [19]:
# Hitung akurasi dari model baseline
correct = 0
for i in range(len(y_test)):
    if (y_predict[i] == ''):
        y_predict[i]  = 'NN'
        
    if y_test[i] == y_predict[i]:
        correct += 1
print('Akurasi Baseline: ',correct / len(y_test) * 100)

Akurasi Baseline:  58.51851851851851
